### Qa) Forklaring til hvad GridSearch er
Den første opgave bestod af at forklare, hvad GridSearch helt konkret er. Der blev givet to kodeceller, hvortil den første skulle beskrives overfladisk, og den anden skulle beskrives mere dybdegående, ved de steder, hvor GridSearch fremtræder.

In [2]:
from time import time
import numpy as np

from sklearn import svm
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn import datasets

from libitmal import dataloaders as itmaldataloaders # Needed for load of iris, moon and mnist

currmode="N/A" # GLOBAL var!

def SearchReport(model): 
    
    def GetBestModelCTOR(model, best_params):
        def GetParams(best_params):
            ret_str=""          
            for key in sorted(best_params):
                value = best_params[key]
                temp_str = "'" if str(type(value))=="<class 'str'>" else ""
                if len(ret_str)>0:
                    ret_str += ','
                ret_str += f'{key}={temp_str}{value}{temp_str}'  
            return ret_str          
        try:
            param_str = GetParams(best_params)
            return type(model).__name__ + '(' + param_str + ')' 
        except:
            return "N/A(1)"
        
    print("\nBest model set found on train set:")
    print()
    print(f"\tbest parameters={model.best_params_}")
    print(f"\tbest '{model.scoring}' score={model.best_score_}")
    print(f"\tbest index={model.best_index_}")
    print()
    print(f"Best estimator CTOR:")
    print(f"\t{model.best_estimator_}")
    print()
    try:
        print(f"Grid scores ('{model.scoring}') on development set:")
        means = model.cv_results_['mean_test_score']
        stds  = model.cv_results_['std_test_score']
        i=0
        for mean, std, params in zip(means, stds, model.cv_results_['params']):
            print("\t[%2d]: %0.3f (+/-%0.03f) for %r" % (i, mean, std * 2, params))
            i += 1
    except:
        print("WARNING: the random search do not provide means/stds")
    
    global currmode                
    assert "f1_micro"==str(model.scoring), f"come on, we need to fix the scoring to be able to compare model-fits! Your scoreing={str(model.scoring)}...remember to add scoring='f1_micro' to the search"   
    return f"best: dat={currmode}, score={model.best_score_:0.5f}, model={GetBestModelCTOR(model.estimator,model.best_params_)}", model.best_estimator_ 

def ClassificationReport(model, X_test, y_test, target_names=None):
    assert X_test.shape[0]==y_test.shape[0]
    print("\nDetailed classification report:")
    print("\tThe model is trained on the full development set.")
    print("\tThe scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, model.predict(X_test)                 
    print(classification_report(y_true, y_pred, target_names))
    print()
    
def FullReport(model, X_test, y_test, t):
    beststr, bestmodel = SearchReport(model)
    ClassificationReport(model, X_test, y_test)    
    print(f"SEARCH TIME: {t:0.2f} sec")
    print(f"CTOR for best model: {bestmodel}\n")
    print(f"{beststr}\n")
    return beststr, bestmodel
    
def LoadAndSetupData(mode, test_size=0.3):
    assert test_size>=0.0 and test_size<=1.0
    
    def ShapeToString(Z):
        n = Z.ndim
        s = "("
        for i in range(n):
            s += f"{Z.shape[i]:5d}"
            if i+1!=n:
                s += ";"
        return s+")"

    global currmode
    currmode=mode
    print(f"DATA: {currmode}..")
    
    if mode=='moon':
        X, y = itmaldataloaders.MOON_GetDataSet(n_samples=5000, noise=0.2)
        itmaldataloaders.MOON_Plot(X, y)
    elif mode=='mnist':
        X, y = itmaldataloaders.MNIST_GetDataSet(load_mode=0)
        if X.ndim==3:
            X=np.reshape(X, (X.shape[0], -1))
    elif mode=='iris':
        X, y = itmaldataloaders.IRIS_GetDataSet()
    else:
        raise ValueError(f"could not load data for that particular mode='{mode}', only 'moon'/'mnist'/'iris' supported")
        
    print(f'  org. data:  X.shape      ={ShapeToString(X)}, y.shape      ={ShapeToString(y)}')

    assert X.ndim==2
    assert X.shape[0]==y.shape[0]
    assert y.ndim==1 or (y.ndim==2 and y.shape[1]==0)    
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=0, shuffle=True
    )
    
    print(f'  train data: X_train.shape={ShapeToString(X_train)}, y_train.shape={ShapeToString(y_train)}')
    print(f'  test data:  X_test.shape ={ShapeToString(X_test)}, y_test.shape ={ShapeToString(y_test)}')
    print()
    
    return X_train, X_test, y_train, y_test

print('OK(function setup, hope MNIST loads works, seem best if you got Keras or Tensorflow installed!)')

OK(function setup, hope MNIST loads works, seem best if you got Keras or Tensorflow installed!)


### Qa) Fortsat:
`SearchReport` - tager en trænet model og printer parametrene fra den bedste model.<br>
`ClassificationReport` - Udskriver information fra Scikit-learns `classification_report` funktion, og gør det ud fra testdataen. <br>
`LoadAndSetupData` - indlæser dataen og og opdeler i train/test-splits. <br>

In [3]:
# TODO: Qa, code review..cell 2) the actual grid-search

# Setup data
X_train, X_test, y_train, y_test = LoadAndSetupData(
    'iris')  # 'iris', 'moon', or 'mnist'

# Setup search parameters
model = svm.SVC(
    gamma=0.001
)  # NOTE: gamma="scale" does not work in older Scikit-learn frameworks,
# FIX:  replace with model = svm.SVC(gamma=0.001)

tuning_parameters = {
    'kernel': ('linear', 'rbf'), 
    'C': [0.1, 1, 10]
}

CV = 5
VERBOSE = 0

# Run GridSearchCV for the model
start = time()
grid_tuned = GridSearchCV(model,
                          tuning_parameters,
                          cv=CV,
                          scoring='f1_micro',
                          verbose=VERBOSE,
                          n_jobs=-1)
grid_tuned.fit(X_train, y_train)
t = time() - start

# Report result
b0, m0 = FullReport(grid_tuned, X_test, y_test, t)
print('OK(grid-search)')

DATA: iris..
  org. data:  X.shape      =(  150;    4), y.shape      =(  150)
  train data: X_train.shape=(  105;    4), y_train.shape=(  105)
  test data:  X_test.shape =(   45;    4), y_test.shape =(   45)


Best model set found on train set:

	best parameters={'C': 1, 'kernel': 'linear'}
	best 'f1_micro' score=0.9714285714285715
	best index=2

Best estimator CTOR:
	SVC(C=1, gamma=0.001, kernel='linear')

Grid scores ('f1_micro') on development set:
	[ 0]: 0.962 (+/-0.093) for {'C': 0.1, 'kernel': 'linear'}
	[ 1]: 0.371 (+/-0.038) for {'C': 0.1, 'kernel': 'rbf'}
	[ 2]: 0.971 (+/-0.047) for {'C': 1, 'kernel': 'linear'}
	[ 3]: 0.695 (+/-0.047) for {'C': 1, 'kernel': 'rbf'}
	[ 4]: 0.952 (+/-0.085) for {'C': 10, 'kernel': 'linear'}
	[ 5]: 0.924 (+/-0.097) for {'C': 10, 'kernel': 'rbf'}

Detailed classification report:
	The model is trained on the full development set.
	The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

           0

C:\Users\thoma\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


### Qa) Fortsat:
Ud fra koden, kan det ses, at dataen først bliver loadet gennem funktionen `LoadAndSetupData`.<br>
Derefter bliver algoritmen `SVC` (Support Vector Classification) valgt. I denne situation er der valgt ikke at prøve flere algoritmer, men det kunne gridsearch også have løst.<br>
Efter dette bliver hyperparametrene bestemt, men modsat i de tidligere opgaver, bliver der nu oprettet en dictionary med lister/tuples af parametre. Dette indikerer hvilke hyperparametrer, som skal prøves i vores gridsearch. `GridSearchCV` modellen bliver derefter oprettet og fitted. Når en `GridSearchCV` model bliver fitted, er det lidt anderledes end det vi er vant til, for den forsøger at fitte med henblik på at finde de bedste hyperparametre. Den gør dette gennem en "brute-force" metode, hvor den prøver alle mulige kombinationer af de hyperparametre, som blev specificeret i vores dictionary.

Hvad betyder det at der er blevet valgt scoringsmetoden `f1_micro`?<br>
Det betyder at vi anvender en `f1_score` til at evaluere modellen men på "micro" niveau. Med dette menes at der kigges på alle klasser samlet i stedet for at score på de enkelte. Dette er fordelagtigt ved iris sættet, da vi har mange klasser, og ønsker at se den totale præstation (ikke med henblik på en enkelt label).


Der blev valgt `n_jobs=-1` til algoritmen. Hvad betyder dette?<br>
`n_jobs` angiver hvor mange tråde, der maksimalt skal dedikeres til at træne modellen. Når den sættes til -1, angiver det, at algoritmen skal bruge så mange tråde den kan. 


### Qb) Erstatning af SVC med SGD
I denne opgave skal `SVC` algoritmen erstattes med en `SGD` algoritme. Der skal derefter foretages en gridsearch, som tager en ikke-ubetydelig mængde tid at eksekvere.

Det ses af kodecellen nedenfor, at algoritmen er erstattet af en `SGDClassifier`. 

TODO: Skriv resultat.

In [36]:
from sklearn.linear_model import SGDClassifier
from threading import Thread
from time import sleep

#def print_time():
#    while(finished == False):
#        print(f"Time since start: {time() - start:.2f}")
#        sleep(10)
        

# Setup data
X_train, X_test, y_train, y_test = LoadAndSetupData(
    'iris')  # 'iris', 'moon', or 'mnist'

# Setup search parameters
model = SGDClassifier()

penalty = ('l1', 'l2')
alpha = np.linspace(start=1E-20, stop=1, num=100)
learning_rate = ['constant', 'optimal', 'invscaling', 'adaptive']
eta0 = np.linspace(start=1E-20, stop=1, num=100)

tuning_parameters = {
    'penalty': penalty, 
    'alpha': alpha,
    'learning_rate': learning_rate,
    'eta0': eta0
}

CV = 5
VERBOSE = 1

# Run GridSearchCV for the model
start = time()
#finished = False
#thread = Thread(target = print_time, args = [])
#thread.start()
grid_tuned = GridSearchCV(model,
                          tuning_parameters,
                          cv=CV,
                          scoring='f1_micro',
                          verbose=VERBOSE,
                          n_jobs=-1)
grid_tuned.fit(X_train, y_train)
t = time() - start
#finished = True

# Report result
b0, m0 = FullReport(grid_tuned, X_test, y_test, t)
print('OK(grid-search)')

: 'constant', 'penalty': 'l2'}
	[79842]: 0.486 (+/-0.378) for {'alpha': 1.0, 'eta0': 0.8080808080808082, 'learning_rate': 'optimal', 'penalty': 'l1'}
	[79843]: 0.695 (+/-0.047) for {'alpha': 1.0, 'eta0': 0.8080808080808082, 'learning_rate': 'optimal', 'penalty': 'l2'}
	[79844]: 0.562 (+/-0.279) for {'alpha': 1.0, 'eta0': 0.8080808080808082, 'learning_rate': 'invscaling', 'penalty': 'l1'}
	[79845]: 0.686 (+/-0.047) for {'alpha': 1.0, 'eta0': 0.8080808080808082, 'learning_rate': 'invscaling', 'penalty': 'l2'}
	[79846]: 0.657 (+/-0.140) for {'alpha': 1.0, 'eta0': 0.8080808080808082, 'learning_rate': 'adaptive', 'penalty': 'l1'}
	[79847]: 0.695 (+/-0.047) for {'alpha': 1.0, 'eta0': 0.8080808080808082, 'learning_rate': 'adaptive', 'penalty': 'l2'}
	[79848]: 0.371 (+/-0.251) for {'alpha': 1.0, 'eta0': 0.8181818181818182, 'learning_rate': 'constant', 'penalty': 'l1'}
	[79849]: 0.333 (+/-0.085) for {'alpha': 1.0, 'eta0': 0.8181818181818182, 'learning_rate': 'constant', 'penalty': 'l2'}
	[79850

## Qc) Tilfældig søgning
I denne opgave udskiftes `GridSearchCV` med den tilfældige søgningsfunktion `RandomizedSearchCV`. Målet er at tjekke om vi opnår at finde en ligestående model ud fra de samme hyperparametre. 

Først skal der gives en forklaring for den nye parametre `n_iter`, som bruges i `RandomizedSearchCV`. `n_iter` er antallet af forskellige kombinationer af de givede hyperparametre. Hvis antaller er 20, ses det at der dannes 20 forskellige modeller af hyperparametrene, som i dette tilfælde krydsvalideres 5 gange (`CV`). Dermed opnås der 100 fits.

In [9]:
from sklearn.linear_model import SGDClassifier
from threading import Thread
from time import sleep

def print_time():
    while(finished == False):
        print(f"Time since start: {time() - start:.2f}")
        sleep(10)
        

# Setup data
X_train, X_test, y_train, y_test = LoadAndSetupData(
    'iris')  # 'iris', 'moon', or 'mnist'

# Setup search parameters
model = SGDClassifier()

penalty = ('l1', 'l2')
alpha = np.linspace(start=1E-20, stop=1, num=30)
learning_rate = ['constant', 'optimal', 'invscaling', 'adaptive']
eta0 = np.linspace(start=1E-20, stop=1, num=30)

tuning_parameters = {
    'penalty': penalty, 
    'alpha': alpha,
    'learning_rate': learning_rate,
    'eta0': eta0
}

CV = 5
VERBOSE = 1

# Run GridSearchCV for the model
start = time()
finished = False
thread = Thread(target = print_time, args = [])
thread.start()
grid_tuned = RandomizedSearchCV(model,
                          tuning_parameters,
                          cv=CV,
                          scoring='f1_micro',
                          n_iter=20,
                          random_state=42,
                          verbose=VERBOSE,
                          n_jobs=-1)
grid_tuned.fit(X_train, y_train)
t = time() - start
finished = True

# Report result
b0, m0 = FullReport(grid_tuned, X_test, y_test, t)
print('OK(grid-search)')

DATA: iris..
  org. data:  X.shape      =(  150;    4), y.shape      =(  150)
  train data: X_train.shape=(  105;    4), y_train.shape=(  105)
  test data:  X_test.shape =(   45;    4), y_test.shape =(   45)

Time since start: 0.00
Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished



Best model set found on train set:

	best parameters={'penalty': 'l1', 'learning_rate': 'optimal', 'eta0': 0.9655172413793103, 'alpha': 0.034482758620689655}
	best 'f1_micro' score=0.9428571428571427
	best index=8

Best estimator CTOR:
	SGDClassifier(alpha=0.034482758620689655, eta0=0.9655172413793103, penalty='l1')

Grid scores ('f1_micro') on development set:
	[ 0]: 0.762 (+/-0.200) for {'penalty': 'l1', 'learning_rate': 'invscaling', 'eta0': 0.5862068965517241, 'alpha': 0.10344827586206896}
	[ 1]: 0.695 (+/-0.047) for {'penalty': 'l1', 'learning_rate': 'adaptive', 'eta0': 0.4482758620689655, 'alpha': 0.7586206896551724}
	[ 2]: 0.695 (+/-0.047) for {'penalty': 'l1', 'learning_rate': 'optimal', 'eta0': 0.7931034482758621, 'alpha': 0.7241379310344828}
	[ 3]: 0.695 (+/-0.047) for {'penalty': 'l2', 'learning_rate': 'adaptive', 'eta0': 0.6206896551724138, 'alpha': 0.7241379310344828}
	[ 4]: 0.695 (+/-0.047) for {'penalty': 'l1', 'learning_rate': 'invscaling', 'eta0': 0.7241379310344828, 

C:\Users\thoma\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


## Qc) Fortsat
Der skal sammenlignes mellem `GridSearchCV` og `RandomizedSearchCV` på både tid og score. 

Tidsmæssigt er der en væsentlig forskel. Den tidligere `GridSearchCV` tog 562.17 sekunder hvorimod `RandomizedSearchCV` kun tog 0.22 sekunder om at finde den bedst mulige model. I tilfælde at datasættet bliver scaleret op og der blivere søgt med flere parametre, vil der kunne spare yderligere tid ved søgning.

Hvis man kigger på scoren af de to fundne modeller ved søgning er der ikke stor forskel. `GridSearchCV` fandt en model med en score på 1.00000 og `RandomizedSearchCV` fandt en model med en 0.94286 score. 

Ved kun at se på disse modeller vil det vurderes at `RandomSearchCV` ser ud til at give en fornuftig model på meget kort tid. Men dette er måske ikke den bedste usecase til at vise hvilken form for søgning er den bedste, da datasættet er forholdvist småt og parametrene få.

Hvis datasættet er enormt og søgningen er med mange parametre, kunne der eventuelt bruges `RandomizedSearchCV` til at lave den første store grove søgning, hvorefter de bedste resultater af den søgning, kunne bruges til at lave en `GridSearchCV` og dermed nå frem til en god model og  endda spare tid.

## Qd) MNIST søgning

Nu skal datasættet ændres til MNIST. Målet med denne opgave er at lave søgninger efter den bedste model til at prædikte MNIST-datasættet. 

Modellerne der valgt i søgningerne er `SDGClassifier`, `RandomForestClassifier` og `GaussianProcessClassifier`. Disse modeller har forskellige parametre som defineres i listen `parameters`. En for-løkke laver så søgninger med hver enkelt model og gemmer den bedste model i listen `b`, som til slut udskrives.



In [ ]:
# Setup data
X_train, X_test, y_train, y_test = LoadAndSetupData(
    'mnist')  # 'iris', 'moon', or 'mnist'

In [ ]:
from sklearn.linear_model import SGDClassifier # <- her
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier 
from IPython.display import clear_output

# Setup search parameters
models = [SGDClassifier(), RandomForestClassifier(), GaussianProcessClassifier()]

almost_zero_to_one=np.linspace(start=1E-20, stop=1, num=30)
zero_to_one=np.linspace(start=0.0, stop=1.0, num=30)
one_to_thousand = np.arange(1, 200)
zero_to_ten = np.arange(0, 10)
one_to_ten = np.arange(1,10)
zero_to_half = np.linspace(start=0, stop=0.5, num=10)
zero_to_hundred = np.arange(0,100)
hundred_to_twohundred = np.arange(100,200)

parameters=[
    {
        #SDG Classifier
        'penalty': ('l1', 'l2'), 
        'alpha': almost_zero_to_one,
        'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
        'eta0': almost_zero_to_one
    },
    {
        #RandomForest
        'n_estimators': one_to_thousand,
        'criterion': ('gini','entropy'),
        'min_samples_split': one_to_ten,
        'min_samples_leaf': zero_to_ten,
        'min_weight_fraction_leaf': zero_to_half,
        'max_features': ('auto','sqrt','log2',None),
        'max_leaf_nodes': [*one_to_thousand, None],
        'class_weight': ('balanced','balanced_subsample')
    },
    {
        #Gaussian Classifier
        'warm_start': (True,False),
        'n_restarts_optimizer': zero_to_hundred,
        'max_iter_predict': hundred_to_twohundred,
        'multi_class': ('one_vs_rest', 'one_vs_one')
    }
]

# tupple
searchables = (models,parameters)

# Reports
b = []
m = []


CV = 5
VERBOSE = 1
globalstart = time()
for i in range(len(searchables[0])):
    # Run search for the model
    localstart = time()
    grid_tuned = RandomizedSearchCV(searchables[0][i],
                                  searchables[1][i],
                                  cv=CV,
                                  scoring='f1_micro',
                                  n_iter=1,
                                  random_state=42,
                                  verbose=VERBOSE,
                                  n_jobs=-1)
    grid_tuned.fit(X_train, y_train)
    t = time() - localstart
    # Report result
    b0, m0 = FullReport(grid_tuned, X_test, y_test, t)
    b.append(b0)
    m.append(m0)
    
t = time() - globalstart
clear_output(wait=True)
print("Tid: " + str(t) +"\n")
print("SGDClassifier: " + b[0] + "\n\nRandomForest: " + b[1] + "\n\n Gaussian Classifier: " + b[2])

DATA: mnist..


C:\Users\thoma\OneDrive - Aarhus Universitet\MAL (Machine Learning)\MAL\ASS03\libitmal\dataloaders.py:65: UserWarning: MNIST_GetDataSet(): failed to import and load data in load_mode 'tensorflow.keras', proceding to next mode..
  warnings.warn("MNIST_GetDataSet(): failed to import and load data in load_mode 'tensorflow.keras', proceding to next mode..")
C:\Users\thoma\OneDrive - Aarhus Universitet\MAL (Machine Learning)\MAL\ASS03\libitmal\dataloaders.py:77: UserWarning: MNIST_GetDataSet(): failed to import and load data in load_mode 'keras', proceding to next mode..
  warnings.warn("MNIST_GetDataSet(): failed to import and load data in load_mode 'keras', proceding to next mode..")
C:\Users\thoma\OneDrive - Aarhus Universitet\MAL (Machine Learning)\MAL\ASS03\libitmal\dataloaders.py:88: UserWarning: MNIST_GetDataSet(): fetch openml mode is slow and uses 'float64' instead of 'uint8'
  warnings.warn("MNIST_GetDataSet(): fetch openml mode is slow and uses 'float64' instead of 'uint8'")
C:\U

  org. data:  X.shape      =(70000;  784), y.shape      =(70000)
  train data: X_train.shape=(49000;  784), y_train.shape=(49000)
  test data:  X_test.shape =(21000;  784), y_test.shape =(21000)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  2.0min remaining:  3.0min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.2min finished



Best model set found on train set:

	best parameters={'penalty': 'l1', 'learning_rate': 'invscaling', 'eta0': 0.5862068965517241, 'alpha': 0.10344827586206896}
	best 'f1_micro' score=0.8179183673469389
	best index=0

Best estimator CTOR:
	SGDClassifier(alpha=0.10344827586206896, eta0=0.5862068965517241,
              learning_rate='invscaling', penalty='l1')

Grid scores ('f1_micro') on development set:
	[ 0]: 0.818 (+/-0.043) for {'penalty': 'l1', 'learning_rate': 'invscaling', 'eta0': 0.5862068965517241, 'alpha': 0.10344827586206896}

Detailed classification report:
	The model is trained on the full development set.
	The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

           0       0.95      0.96      0.95      2077
           1       0.91      0.94      0.92      2385
           2       0.90      0.82      0.86      2115
           3       0.87      0.82      0.84      2117
           4       0.90      0.84      0.87     

C:\Users\thoma\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    3.9s remaining:    6.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    4.0s finished



Best model set found on train set:

	best parameters={'n_estimators': 37, 'min_weight_fraction_leaf': 0.4444444444444444, 'min_samples_split': 7, 'min_samples_leaf': 5, 'max_leaf_nodes': 187, 'max_features': 'log2', 'criterion': 'entropy', 'class_weight': 'balanced_subsample'}
	best 'f1_micro' score=0.6014081632653061
	best index=0

Best estimator CTOR:
	RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy',
                       max_features='log2', max_leaf_nodes=187,
                       min_samples_leaf=5, min_samples_split=7,
                       min_weight_fraction_leaf=0.4444444444444444,
                       n_estimators=37)

Grid scores ('f1_micro') on development set:
	[ 0]: 0.601 (+/-0.049) for {'n_estimators': 37, 'min_weight_fraction_leaf': 0.4444444444444444, 'min_samples_split': 7, 'min_samples_leaf': 5, 'max_leaf_nodes': 187, 'max_features': 'log2', 'criterion': 'entropy', 'class_weight': 'balanced_subsample'}

Detailed classification rep

C:\Users\thoma\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


              precision    recall  f1-score   support

           0       0.51      0.90      0.65      2077
           1       0.56      0.96      0.71      2385
           2       0.74      0.39      0.51      2115
           3       0.71      0.60      0.65      2117
           4       0.50      0.79      0.61      2004
           5       0.74      0.04      0.07      1900
           6       0.66      0.53      0.59      2045
           7       0.61      0.73      0.67      2189
           8       0.61      0.60      0.60      2042
           9       0.77      0.27      0.40      2126

    accuracy                           0.59     21000
   macro avg       0.64      0.58      0.55     21000
weighted avg       0.64      0.59      0.55     21000


SEARCH TIME: 6.28 sec
CTOR for best model: RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy',
                       max_features='log2', max_leaf_nodes=187,
                       min_samples_leaf=5, min_samples

[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  2.1min remaining:  3.1min
